In [1]:
import pandas as pd
import numpy as np
import ast
import glob

In [2]:
# Configuración
trajectory_length = 48
seconds_in_a_half_day = 24 * 60 * 60 / 2
intervals_per_day = seconds_in_a_half_day // trajectory_length

start_time_limit = 3600 * 6  # 6:00 AM
end_time_limit = 3600 * 18  # 6:00 PM

In [3]:
import pandas as pd

def process_file(file_path):
    # Leer el archivo de texto y extraer las líneas
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Saltar la primera línea y eliminar espacios en blanco
    lines = [line.strip().strip('"') for line in lines[1:]]

    # Lista para almacenar los resultados procesados
    processed_rows = []

    for i, line in enumerate(lines):
        parts = line.split("],[")
        
        # Asegurar que los corchetes se mantienen
        zones = ast.literal_eval(parts[0] + "]")      # Añadimos el corchete final
        time_start = ast.literal_eval("[" + parts[1] + "]") # Añadimos el corchete inicial y final
        time_end = ast.literal_eval("[" + parts[2])   # Añadimos el corchete inicial

        # Procesar cada elemento y agregarlo a `processed_rows`
        processed_rows.extend([
            {
                'zone': zone,
                'time_start': start,
                'time_end': end,
                'user_index': i
            }
            for zone, start, end in zip(zones, time_start, time_end)
            if zone is not None  # Ignorar valores nulos
        ])

    # Crear un DataFrame con los datos procesados
    return pd.DataFrame(processed_rows)


In [4]:
def process_all_files(folder_path):
    # Procesar todos los archivos .txt en la carpeta
    all_files = glob.glob(f"{folder_path}/*.txt")
    total_files = len(all_files)
    
    # Lista para almacenar los DataFrames procesados
    data_frames = []
    
    for i, file in enumerate(all_files):
        # Procesar el archivo
        df = process_file(file)
        
        # Agregar la columna 'day' al DataFrame procesado
        df['day'] = i
        
        # Añadir a la lista de DataFrames
        data_frames.append(df)
        print(f"[{i+1}/{total_files}] Archivo procesado con éxito: {file}")
    
    # Concatenar todos los DataFrames en uno solo
    return pd.concat(data_frames, ignore_index=True)

In [9]:
def expand_intervals(data):
    # Calcular los índices `t_start` y `t_end` para todo el DataFrame
    t_start = (data['time_start'] // intervals_per_day).astype(int)
    t_end = (data['time_end'] // intervals_per_day).astype(int)

    # Calcular la longitud de cada intervalo para todos los registros
    lengths = t_end - t_start + 1

    # Crear un índice repetido para expandir los datos
    expanded_indices = np.repeat(data.index, lengths)

    # Generar valores `t` para todos los intervalos de una vez
    expanded_t = np.concatenate([np.arange(start, end + 1) for start, end in zip(t_start, t_end)])

    # Crear el DataFrame expandido
    expanded_data = pd.DataFrame({
        'uid': data.loc[expanded_indices, 'user_index'].values,
        'd': data.loc[expanded_indices, 'day'].values,
        't': expanded_t,
        'p_i': data.loc[expanded_indices, 'p_i'].values
    })

    return expanded_data

In [13]:
def transform_to_trajectory(data):
    # Crear una columna adicional para almacenar `p_i` (si es necesario)
    data['p_i'] = data['zone']

    # Expandir los intervalos una sola vez en un DataFrame directamente
    expanded_df = pd.DataFrame(expand_intervals(data))

    # Crear listas para trayectorias finales
    trajectories = []
    user_indices = []
    days = []

    # Agrupar por `uid` y `d` utilizando `groupby`
    grouped = expanded_df.groupby(['uid', 'd'])

    for (uid, day), group in grouped:
        # Inicializar la trayectoria completa con '[PAD]'
        trajectory = np.full(trajectory_length, '[PAD]', dtype=object)

        # Optimizar el cálculo del valor más frecuente por `t`
        t_p_i_grouped = group.groupby('t')['p_i'].agg(lambda x: x.mode()[0])

        # Convertir t_p_i_grouped a índices y valores
        indices = np.array(t_p_i_grouped.index)
        values = np.array(t_p_i_grouped.values, dtype=str)

        if len(indices) < 7:
            continue 

        trajectory[indices] = values

        # Agregar a las listas finales
        trajectories.append(trajectory)
        user_indices.append(uid)
        days.append(day)

    # Crear DataFrame con las trayectorias finales
    result_df = pd.DataFrame({
        'trajectory': trajectories,
        'user_index': user_indices,
        'day': days
    })

    return result_df

In [7]:
raw_data = pd.read_csv("../data/etecsa/etecsa_processed.csv")
raw_data = raw_data[(raw_data['time_start'] >= start_time_limit) & (raw_data['time_end'] <= end_time_limit)]
raw_data['time_start'] = raw_data['time_start'] - start_time_limit
raw_data['time_end'] = raw_data['time_end'] - start_time_limit

In [ ]:
data = transform_to_trajectory(raw_data)
print(data.count())

In [ ]:
# Guardar como HDF5
h5_file = "etecsa.h5"
data.to_hdf(h5_file, key='data', mode='w')